In [1]:
import pulp

p.17

In [2]:
problem = pulp.LpProblem("MANUFACTURE", pulp.LpMaximize)

p = pulp.LpVariable("p", cat="Continuous")
q = pulp.LpVariable("q", cat="Continuous")

problem += 1*p+3*q<=30
problem += 2*p+1*q<=40
problem += p>=0
problem += q>=0
problem += 2*p+1*q

status = problem.solve()

print('Status:', pulp.LpStatus[status])
print('p=', p.value(), 'q=', q.value(), "obg=", problem.objective.value())

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/yusuke/Documents/PyOptBook/env/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/j9/hc7q_ty53kb29vw6s6nxzdlh0000gn/T/69ebc044f8dd43e180cdbf2f0c6c0cae-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/j9/hc7q_ty53kb29vw6s6nxzdlh0000gn/T/69ebc044f8dd43e180cdbf2f0c6c0cae-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 9 COLUMNS
At line 18 RHS
At line 23 BOUNDS
At line 26 ENDATA
Problem MODEL has 4 rows, 2 columns and 6 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 2 (-2) rows, 2 (0) columns and 4 (-2) elements
0  Obj -0 Dual inf 2.999998 (2)
2  Obj 40
Optimal - objective value 40
After Postsolve, objective 40, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 40 - 2 iterations time 0.002, Presolve 0.00
Option for printingOptions changed 

## 定式化
### リスト，定数，変数，制約式，目的関数を順に定めて，プログラムする．

p.23

In [3]:
import pandas as pd
from pathlib import Path

In [9]:
# set path
stock_path = Path("../2.tutorial/stocks.csv")
gain_path = Path("../2.tutorial/gains.csv")
requires_path = Path("../2.tutorial/requires.csv")

# load data
stock_df = pd.read_csv(stock_path)
gain_df = pd.read_csv(gain_path)
require_df = pd.read_csv(requires_path)

In [5]:
stock_df.head()

,m,stock
0,m1,35
1,m2,22
2,m3,27


In [6]:
gain_df.head()

,p,gain
0,p1,3
1,p2,4
2,p3,4
3,p4,5


In [7]:
require_df.head()

,p,m,require
0,p1,m1,2
1,p1,m2,0
2,p1,m3,1
3,p2,m1,3
4,p2,m2,2


In [12]:
# リストの定義
# 製品のリスト P
P = gain_df["p"].tolist()
# 材料のリスト
M = stock_df["m"].tolist()

print(P)
print(M)

['p1', 'p2', 'p3', 'p4']
['m1', 'm2', 'm3']


In [18]:
# 定数の定義
stock = {row.m:row.stock for row in stock_df.itertuples()}
print(stock)
gain = {row.p:row.gain for row in gain_df.itertuples()}
print(gain)
require = {(row.p, row.m):row.require for row in require_df.itertuples()}
print(require)

{'m1': 35, 'm2': 22, 'm3': 27}
{'p1': 3, 'p2': 4, 'p3': 4, 'p4': 5}
{('p1', 'm1'): 2, ('p1', 'm2'): 0, ('p1', 'm3'): 1, ('p2', 'm1'): 3, ('p2', 'm2'): 2, ('p2', 'm3'): 0, ('p3', 'm1'): 0, ('p3', 'm2'): 2, ('p3', 'm3'): 2, ('p4', 'm1'): 2, ('p4', 'm2'): 2, ('p4', 'm3'): 2}


In [19]:
# define problem
problem = pulp.LpProblem("some_variables_problem", 
                         sense=pulp.LpMaximize)

In [22]:
# 変数の定義 dictsで一気に定義
x = pulp.LpVariable.dicts("x", P, cat='Continuous')
print(x)

{'p1': x_p1, 'p2': x_p2, 'p3': x_p3, 'p4': x_p4}


In [23]:
# 制約式の定義
for p in P:
    problem += x[p] >= 0

In [26]:
for m in M:
    problem += pulp.lpSum([require[p, m]*x[p] for p in P]) <= stock[m]

In [30]:
# 目的関数の定義
problem += pulp.lpSum([gain[p]*x[p] for p in P])

In [33]:
status = problem.solve()
print(problem)
print("Status : ", pulp.LpStatus[status])

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/yusuke/Documents/PyOptBook/env/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/j9/hc7q_ty53kb29vw6s6nxzdlh0000gn/T/c6b1c5fc117f4712bb9a0f8b21bce1a2-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/j9/hc7q_ty53kb29vw6s6nxzdlh0000gn/T/c6b1c5fc117f4712bb9a0f8b21bce1a2-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 12 COLUMNS
At line 30 RHS
At line 38 BOUNDS
At line 43 ENDATA
Problem MODEL has 7 rows, 4 columns and 13 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 3 (-4) rows, 4 (0) columns and 9 (-4) elements
0  Obj -0 Dual inf 17.499996 (4)
4  Obj 80.428571
Optimal - objective value 80.428571
After Postsolve, objective 80.428571, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 80.42857143 - 4 iterations time 0.002, Presolve 0.00
Op

In [34]:
for p in P:
    print(p, x[p].value())

p1 12.142857
p2 3.5714286
p3 7.4285714
p4 0.0


In [35]:
# 変数を整数値として定義したい場合はLpVariableでcat="Integer"を用いる．

# 数理最適化モデルの定義
problem = pulp.LpProblem('LP2', pulp.LpMaximize)

# 変数の定義
x = pulp.LpVariable.dicts('x', P, cat='Integer')

# 制約式の定義
for p in P:
    problem += x[p] >= 0
for m in M:
    problem += pulp.lpSum([require[p,m] * x[p] for p in P]) <= stock[m]

# 目的関数の定義    
problem += pulp.lpSum([gain[p] * x[p] for p in P])

# 求解
status = problem.solve()
print('Status:', pulp.LpStatus[status])

# 計算結果の表示
for p in P:
    print(p, x[p].value())

print('obj=', problem.objective.value())

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/yusuke/Documents/PyOptBook/env/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/j9/hc7q_ty53kb29vw6s6nxzdlh0000gn/T/db9d6b2d949749f7b383216a90728b78-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/j9/hc7q_ty53kb29vw6s6nxzdlh0000gn/T/db9d6b2d949749f7b383216a90728b78-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 12 COLUMNS
At line 38 RHS
At line 46 BOUNDS
At line 51 ENDATA
Problem MODEL has 7 rows, 4 columns and 13 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 80.4286 - 0.00 seconds
Cgl0004I processed model has 3 rows, 4 columns (4 integer (0 of which binary)) and 9 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0012I Integer solution of -76 found by DiveCoefficient after 0 iterations and 0 nodes (0.01 second